In [24]:
import math
import re
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd

# helper functions

def parse_coordinates(coord_string):
    """
    Parse the latitude and longitude coordinates in the format: "24°29′23.7″N 120°40′16″E".
    Returns latitude and longitude in decimal degrees.
    """
    pattern = r'(\d+)°(\d+)′([\d.]+)″([NS])\s+(\d+)°(\d+)′([\d.]+)″([EW])'
    match = re.match(pattern, coord_string)
    
    if match:
        lat_deg = int(match.group(1))
        lat_min = int(match.group(2))
        lat_sec = float(match.group(3))
        lat_dir = match.group(4)
        lon_deg = int(match.group(5))
        lon_min = int(match.group(6))
        lon_sec = float(match.group(7))
        lon_dir = match.group(8)
        
        lat_decimal = lat_deg + lat_min/60 + lat_sec/3600
        if lat_dir == 'S':
            lat_decimal = -lat_decimal
            
        lon_decimal = lon_deg + lon_min/60 + lon_sec/3600
        if lon_dir == 'W':
            lon_decimal = -lon_decimal
            
        return lat_decimal, lon_decimal
    
    else:
        return None

def parse_coordinates_pair(coords_str):  
    latitude, longitude = parse_coordinates(coords_str)
    return np.array((latitude, longitude))

def calculate_distance(coords_1, coords_2):
    """
    Calculate the distance between two coordinates using the Haversine formula.
    """
    # print(coords_1, coords_2)
 
    lat1, lon1 = coords_1
    lat2, lon2 = coords_2
    
    
    # Convert latitude and longitude from degrees to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)
    
    # Haversine formula
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    a = math.sin(dlat/2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = 6371 * c  # Radius of the Earth in kilometers
    
    return distance

def find_nearest_location(target_coords, coordinates):
    """
    Find the nearest location to the target coordinate from a set of coordinates.
    """
        
    min_distance = float('inf')
    nearest_location = None
    
    for coords in coordinates:
        distance = calculate_distance(target_coords, coords)
        if distance < min_distance:
            min_distance = distance
            nearest_location = coords
    
    return nearest_location, min_distance

def get_target_distance(coords_series, target_coord):
    distance = np.zeros_like(coords_series.to_numpy(copy=True))
    for i, c in enumerate(coords_series):
        distance[i] = calculate_distance(c, target_coord)

    return distance

# # Set of coordinates (latitude, longitude)
# coordinates = [
#     (37.7749, -122.4194),  # San Francisco
#     (34.0522, -118.2437),  # Los Angeles
#     (40.7128, -74.0060),   # New York
#     (51.5074, -0.1278)     # London
# ]

# # Target coordinate (latitude, longitude)
# target_coords = (51.5074, -0.1278)
# target_lat = 51.5074
# target_lon = -0.1278  # Chicago

# # Find the nearest location
# # nearest_location, distance = find_nearest_location(target_lat, target_lon, coordinates)
# nearest_location, distance = find_nearest_location(target_coords, coordinates)

# print("Nearest Location:", nearest_location)
# print("Distance:", distance, "km")

In [25]:
# load station / power plant data
station_pos = pd.read_csv('.\\Station Locations.csv')
station_pos['Parsed Coords'] = station_pos.apply(lambda row: np.array((row['"twd97lat"'], row['"twd97lon"'])), axis=1)
plant_pos = pd.read_csv('.\\Power Plant Locations.csv')
plant_pos['Parsed Coords'] = plant_pos['Coordinates'].apply(parse_coordinates_pair)

# find the cloest station for each power plant

station_included = set()
plant_stations = {}

for i, plant in plant_pos.iterrows():
    station_distance = get_target_distance(station_pos['Parsed Coords'], plant['Parsed Coords'])
    station_nearby = np.argmin(station_distance)
    station_included.add(station_pos['"siteid"'][station_nearby])
    plant_stations[plant['Chinese']] = set()
    plant_stations[plant['Chinese']].add(station_pos['"siteid"'][station_nearby])


,"""sitename""","""siteengname""","""areaname""","""county""","""township""","""siteaddress""","""twd97lon""","""twd97lat""","""sitetype""","""siteid""",Parsed Coords
54,前金,Qianjin,高屏空品區,高雄市,前金區,高雄市三民區市中一路339號,120.286761,22.633903,一般站,56,"[22.63390278, 120.28676111]"
69,復興,Fuxing,高屏空品區,高雄市,前鎮區,高雄市前鎮區民權二路331號,120.312017,22.608711,交通站,71,"[22.608711, 120.312017]"
55,前鎮,Qianzhen,高屏空品區,高雄市,前鎮區,高雄市前鎮區中山三路43號,120.308334,22.604451,工業站,57,"[22.6044507, 120.30833356]"
53,左營,Zuoying,高屏空品區,高雄市,左營區,高雄市左營區翠華路687號,120.292917,22.674861,一般站,54,"[22.674861, 120.292917]"
49,鳳山,Fengshan,高屏空品區,高雄市,鳳山區,高雄市鳳山區曹公路6號,120.357422,22.628126,交通站,50,"[22.628126, 120.357422]"
